# A full business solution

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

In [1]:
# imports

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [2]:
# Initialize and constants

load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'your-key-if-not-using-env')
MODEL = 'gpt-4o-mini'
openai = OpenAI()

In [3]:
# A class to represent a Webpage

class Website:
    url: str
    title: str
    body: str
    links: List[str]

    def __init__(self, url):
        self.url = url
        response = requests.get(url)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [6]:
ed = Website("https://edwarddonner.com")
# print(ed.links)
print(ed.get_contents())

Webpage Title:
Home - Edward Donner
Webpage Contents:
Home
Outsmart
An arena that pits LLMs against each other in a battle of diplomacy and deviousness
About
Posts
Well, hi there.
I’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (
very
amateur) and losing myself in
Hacker News
, nodding my head sagely to things I only half understand.
I’m the co-founder and CTO of
Nebula.io
. We’re applying AI to a field where it can make a massive, positive impact: helping people discover their potential and pursue their reason for being. Recruiters use our product today to source, understand, engage and manage talent. I’m previously the founder and CEO of AI startup untapt,
acquired in 2021
.
We work with groundbreaking, proprietary LLMs verticalized for talent, we’ve
patented
our matching model, and our award-winning platform has happy customers and tons of pr

## First step: Have GPT-4o-mini figure out which links are relevant

### Use a call to gpt-4o-mini to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

In [7]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""

In [8]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [9]:
print(get_links_user_prompt(ed))

Here is the list of links on the website of https://edwarddonner.com - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
https://edwarddonner.com/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://patents.google.com/patent/US20210049536A1/
https://www.linkedin.com/in/eddonner/
https://edwarddonner.com/2024/08/06/outsmart/
https://edwarddonner.com/2024/08/06/outsmart/
https://edwarddonner.com/2024/06/26/choosing-the-right-llm-resources/
https://edwarddonner.com/2024/06/26/choosing-the-right-llm-resources/
https

In [10]:
def get_links(url):
    website = Website(url)
    completion = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = completion.choices[0].message.content
    return json.loads(result)

In [13]:
get_links("https://anthropic.com")

{'links': [{'type': 'about page', 'url': 'https://anthropic.com/company'},
  {'type': 'careers page', 'url': 'https://anthropic.com/careers'},
  {'type': 'team page', 'url': 'https://anthropic.com/team'},
  {'type': 'research page', 'url': 'https://anthropic.com/research'},
  {'type': 'news page', 'url': 'https://anthropic.com/news'}]}

## Second step: make the brochure!

Assemble all the details into another prompt to GPT4-o

In [14]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [15]:
print(get_all_details("https://anthropic.com"))

Found links: {'links': [{'type': 'about page', 'url': 'https://anthropic.com/company'}, {'type': 'careers page', 'url': 'https://anthropic.com/careers'}, {'type': 'team page', 'url': 'https://anthropic.com/team'}, {'type': 'research page', 'url': 'https://anthropic.com/research'}, {'type': 'enterprise page', 'url': 'https://anthropic.com/enterprise'}, {'type': 'api page', 'url': 'https://anthropic.com/api'}]}
Landing page:
Webpage Title:
Home \ Anthropic
Webpage Contents:
Claude
Overview
Team
Enterprise
API
Pricing
Research
Company
Careers
News
AI
research
and
products
that put safety at the frontier
New
Meet Claude 3.5 Sonnet
Claude 3.5 Sonnet, our most intelligent AI model, is now available.
Talk to Claude
API
Build with Claude
Start using Claude to drive efficiency and create new revenue streams.
Get started now
Our Work
Announcements
Claude 3.5 Sonnet
Jun 21, 2024
Alignment
·
Research
Constitutional AI: Harmlessness from AI Feedback
Dec 15, 2022
Announcements
Core Views on AI Safet

In [16]:
# system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."

In [26]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

# Or uncomment the line below for a more humorous brochure:

# system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."


In [17]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:20_000] # Truncate if more than 20,000 characters
    return user_prompt

In [18]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [20]:
create_brochure("Anthropic", "https://anthropic.com")

Found links: {'links': [{'type': 'about page', 'url': 'https://anthropic.com/company'}, {'type': 'careers page', 'url': 'https://anthropic.com/careers'}, {'type': 'research page', 'url': 'https://anthropic.com/research'}, {'type': 'team page', 'url': 'https://anthropic.com/team'}]}


# Welcome to Anthropic: Where AI Gets the VIP Treatment!

## 🚀 Meet Claude: Your New AI BFF 🤖
Introducing **Claude 3.5 Sonnet**! Our most intelligent AI model that’s not just smart, it’s poetic! Think of it as your personal assistant that can draft emails, recover lost socks, and maybe even recite Shakespeare – or at least summarize it for you. 

## 🌟 Our Mission: Safety First!
At Anthropic, we're on a mission to make AI systems that are reliable, interpretable, and steerable. We believe that AI should help people—not just make your computer crash or start a dramatic sci-fi movie plot.

> *"Why did the robot go on a diet? To reduce its bytes!"*

## 🤝 Company Culture: Kindness is Key
At Anthropic, we keep trust levels high! Our culture promotes kindness, honesty, and emotional maturity. So, if you’ve ever wanted to have a disagreement about AI safety while sipping tea (or kombucha—you do you), this is the place for you!

## 🧑‍🔬 Our Interdisciplinary Team: All Mattresses, No Topper
Our crew is like a buffet of expertise—researchers, engineers, policy experts, and operations leaders, all gathering around the table of innovation. We even embrace an open-door policy (unless it’s a safety door—safety first, remember?)!

## 🌍 Customers Big and Small
We work with enterprises, nonprofits, and civil society groups, ensuring that everyone from tech giants to your local bake sale can rely on our AI. After all, even your grandma can benefit from AI in her next baking project, right?

## 🥳 Careers at Anthropic: Join the AI Safety Party!
We’re always hunting for fresh talent. Our perks include generous parental leave, unlimited PTO (seriously!), and a secret stash of dad jokes. If you're interested in being part of a team that promises not to blow up the world (or your computer), check out our open roles!

> *"What do you call an AI that tells jokes? A-comedian!"*

## 🎉 Why Work Here?
- **Unlimited PTO**: Even plants get a day off!
- **Wellness Stipend**: We want your mind and body firing on all cylinders!
- **Diverse Opportunities**: From engineering to policy work, we see all talents as essential.
- **Remote Flexibility**: Work wherever you flip your laptop open (just be sure there's a Wi-Fi signal).

## 🌱 Bringing It All Together
At Anthropic, we understand that the future of AI depends on safety and collaboration. We are committed to making this tech world not just smarter, but also a better place for everyone. Join us in crafting the future—preferably one that includes less drama and more happy endings!

### Ready to Join the Adventure?
Check out [Careers at Anthropic](#) to find your position in our AI utopia! 

---

**Anthropic: Where Your Talks with Claude Matter (but don't worry, we won't tell!)**

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [21]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [22]:
stream_brochure("Anthropic", "https://anthropic.com")

Found links: {'links': [{'type': 'about page', 'url': 'https://anthropic.com/company'}, {'type': 'careers page', 'url': 'https://anthropic.com/careers'}, {'type': 'team page', 'url': 'https://anthropic.com/team'}, {'type': 'research page', 'url': 'https://anthropic.com/research'}, {'type': 'news page', 'url': 'https://anthropic.com/news'}]}



# Welcome to Anthropic: Home of Claude & Safety First AI!

## Who Are We?
Welcome to Anthropic, where we believe AI should be as safe as a toddler in a playground (well, at least as safe as we can make it)! Based in the picturesque *San Francisco*, we’re a **Public Benefit Corporation** committed to building reliable, interpretable, and steerable AI systems. Think of us as the safety net that makes all those futuristic dreams of AI come true—without the terrifying nightmares!

## Meet Claude: Your New Best Friend in AI!
Say hello to **Claude 3.5 Sonnet**—our resident genius and the brightest star in our AI constellation! Whether you need help with generating code, drafting job descriptions, or just need a buddy for brainstorming, Claude is here to help. Think of Claude as that hyper-intelligent friend who always has the right answer (but way less judgmental).

### Fun Facts About Claude:
- Can whip up a to-do list faster than you can say “artificial intelligence.”
- Has been known to significantly reduce email response times—goodbye inbox anxiety!
- Just don’t ask him to pick a favorite movie; we’re still working on his *emotional intelligence*.

## Our Culture: Where Teamwork Makes the Dream Work!
At Anthropic, we're not just a bunch of techies in hoodies. We pride ourselves on our **collaborative culture** built on trust. Our values include:
- **Mission-Driven**: Everyone here is on a mission to make transformative AI help society flourish. So you can bet we’re not just googling “How to make AI safe” in our spare time!
- **Unusually High Trust**: We trust each other like we trust a cat to not knock over your favorite plant (spoiler: never).
- **One Big Team**: We’re all in this together, like the Avengers but with more laptops and fewer capes.
  
If you love a good group project or can code a Python script in your sleep, you’ll fit right in!

## Careers: Join the Safety Squad!
Looking for a job that combines your love for AI with a mission that actually matters? Look no further! We’re always on the lookout for passionate minds in roles ranging from research, engineering to operations. 
- **Perks? You Bet!** 
    - **Unlimited PTO**: Because sometimes you just need to go watch your favorite show—Netflix needs you.
    - **Health & Wellness Benefits**: We care about you like your mom does. 
    - **Remote and Flexible Work**: If you prefer to work while wearing pajamas (we get it) or buzzing into the office, we've got you covered.

So, if you’ve got skills as impressive as Claude's conversational abilities, check out our [open roles](#) and apply! 

## Our Customers: Who's Using Claude?
From startups to large enterprises, our customers are revolutionizing how they work with the help of Claude. Clubs, offices, and perhaps a pirate ship or two (but we're still waiting on confirmation). Don't just take our word for it—stress-free productivity is *the* new norm!

### 🎉 Testimonials: 
"Claude saved us hours of work—it’s like having a personal assistant without the coffee runs!" – A very happy customer (who wishes to remain anonymous).

> "With Claude, we went from zero to project heroes in a matter of days!" – Another satisfied customer. 

## Join Us!
So, whether you're looking for reliable AI systems or a workplace that doubles as the coolest club ever, Anthropic welcomes you! Let’s build a safer AI world together—one *Claude*-sized leap at a time!

Come explore the funny side of AI at [Anthropic](#) today!



In [23]:
stream_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'blog', 'url': 'https://huggingface.co/blog'}, {'type': 'company page', 'url': 'https://www.linkedin.com/company/huggingface/'}, {'type': 'public discussion forum', 'url': 'https://discuss.huggingface.co'}]}


# Welcome to Hugging Face: Where AI Gets a Hug!

## 🤗 Embrace the Future of Machine Learning! 

### Who Are We?
We are **Hugging Face**, an AI community on a mission to democratize machine learning, one heartwarming commit at a time. With over **400k models**, **150k applications**, and **100k datasets** to choose from, we’ve got everything you need to make your ML dreams come true (no magic wand required!).

### What We Offer
- **Models**: Choose from the latest and greatest models like **Llama-3.2** and the well-anticipated **GOT-OCR**. Whether you want to feign intelligence or genuinely spark it, we have you covered!
- **Datasets**: Dive into a sea of over **100,000 datasets**. It's like a buffet, but with data instead of food—no calories, just algorithms.
- **Spaces**: Create, collaborate, and show off your projects! (You make it look easy, you’re the star of our space!)

### Our Customers 💡
We’re not just the life of the algorithm party; we’re a favorite of over **50,000 organizations** including tech giants like **Meta**, **Google**, and **Microsoft**. No biggie, right? Join the ranks of these illustrious names and leave your mark on our community!

### Career at Hugging Face 🎉
Thinking of joining us? We’re always on the lookout for the next great talent! At Hugging Face, our **culture** is about collaboration, innovation, and the occasional meme battle. If you can code and make everyone laugh at the same time, you might be our kind of person!

- **Open Positions**: From machine learning researchers to remote cuddly developers—there’s something for everyone. Plus, who wouldn’t want to work at a place with the coolest name? 

### Noteworthy Features 
- **Transformers**: Our state-of-the-art library that makes ML look as easy as pie (which we might also include if you'd like).
- **HuggingChat**: Now with AI tools, it’s like having a chat with a nerdy friend who knows everything!
- **Compute Solutions**: Need more power? We can configure a GPU in your sleep—well, almost!

### Join Us! 
Let’s shape the future of AI together! Sign up today, and who knows, you might end up making the next big model that shakes the tech world (or at least earns you some serious bragging rights).

### A Little Humor 🐵 
Why did the dataset break up with the model?   
Because **it just couldn't handle the weight of their relationship!** 

---

Ready to take the plunge? Visit us at **[Hugging Face](https://huggingface.co)** – where coding meets creativity, and every model is hugged! 

In [24]:
stream_brochure("Oracle", "https://www.oracle.com/")

Found links: {'links': [{'type': 'about page', 'url': 'https://www.oracle.com/'}, {'type': 'customers page', 'url': 'https://www.oracle.com/customers'}, {'type': 'accessibility page', 'url': 'https://www.oracle.com/corporate/accessibility/'}]}


# 🥳 Welcome to Oracle: The Cloud's Funhouse! ☁️💼

## Who Are We? 🤔
At Oracle, we’re not just spinning in the cloud; we’re flying with style! From supply chains to superheroes (also known as our engineers), we help companies of all shapes and sizes find their inner tech titan. Our motto? **"Data is like air: everywhere, essential, and sometimes a bit out of reach!"**

## Our Magic Wand: Oracle Cloud! 🪄
- **Cloud Applications & Platforms** 🖥️: Transform your business with our cloud solutions faster than you can say "XML!"
- **AI & Automation** 🤖: Because we all need an assistant... or two! We've got AI features that are so smart; they might just outsmart you. (Just kidding. We love you!)
- **Fusion Cloud Applications** 💻: Our secret sauce that helps businesses like **DHL**, **AMC Theatres**, and **Hyatt** manage everything from their supply chain to their snack inventories.

## 🚀 Customers Love Us!
With over **400,000 customers**, it's safe to say we're kind of a big deal. Here’s what some of them are saying:
- **Uber**: “Thanks to Oracle, we know how to manage logistics better than we know how to pronounce ‘logistics!’”
- **MGM Resorts**: “Oracle helped us impress guests so well, they leave with smiles bigger than the strip!”

## 🎉 Culture at Oracle
At Oracle, we value creativity and innovation more than we value that leftover coffee from last week. But don’t worry, we’re *still* committed to maintaining a healthy work-life balance. **Perks include**:
- **Friendly Competition**: From hackathons to pizza-eating contests (yes, those are real), we encourage teamwork and a little bit of rivalry. 
- **Work Remote or On-site**: If your cat is your ideal coworker, we totally get it!
- **Diversity and Inclusion**: We're committed to creating accessible technologies. Everyone is welcome, especially if you appreciate a good dad joke!

## 💼 Careers: Join the Fun!
Want to work in a place where the cloud doesn’t mean gloomy weather? Check out our job openings that span from developers to cloud wizards:
- **Job Perks**: Not just competitive salaries, but also an environment so lively, you might forget you’re at work!
- **Open Positions**: Software engineers, cloud architects, coffee enthusiast (seriously, we need more of that).

## 🏆 Oracle: The Cloud with a Sense of Humor!
Join us as we revolutionize businesses and sprinkle a little joy along the way. Whether you’re looking to partner with us, explore career options, or simply learn about cloud magic, Oracle is the place where dreams and data come true!

## Call to Action! 📞
Feeling inspired? Visit our website now and experience the **Oracle Cloud magic!** 🌟

---

*Oracle: Because every cloud has a silver lining, especially when it’s ours!*

In [25]:
stream_brochure("Worley", "https://www.worley.com/")

Found links: {'links': [{'type': 'about page', 'url': 'https://www.worley.com/en/about-us'}, {'type': 'careers page', 'url': 'https://www.worley.com/en/careers'}, {'type': 'investor relations page', 'url': 'https://www.worley.com/en/investor-relations'}, {'type': 'company page', 'url': 'https://www.worley.com/en/about-us/our-story'}, {'type': 'contact page', 'url': 'https://www.worley.com/en/contact-us'}, {'type': 'sustainability page', 'url': 'https://www.worley.com/en/sustainability'}]}


# Welcome to Worley: The World’s Most Sustainable Comedy Routine! 🌎✨

---

### Why Worley? Well, Let’s Break it Down! 

Did you lose your way trying to solve the world's energy crisis? No worries! At Worley, we’re bridging two worlds—traditional energy and sustainable solutions—quicker than you can say "carbon-neutral!" We’re nearly *50,000* experts in this 45-country dance-off of energy transformation, working together to bring you the land of *sustainable delights*!

---

### Here’s What We Do (and How We Make Fun of It!)

- **Chemicals & Fuels**: We help refine not only gasoline but also your knowledge on low-carbon solutions! It's like being a barista but for *biodiesel*! ☕

- **Energy Storage & Hydrogen**: You need somewhere to store all that hot air? We’ve got it covered—just ask our hydrogen hubs! (No attachment lifting required!)

- **Resources**: From precious metals to aluminum for your next rocket project (or just a soda can), we got you sorted! We’re basically the *Costco* of sustainability!

---

### Our Culture: A Team of Eco-Warriors with a Sense of Humor! 🦸‍♂️🦸‍♀️

At Worley, our values guide us like the best GPS:

- **We Value Life**: Because after all, "no humans, no fun!"  
- **We Rise to the Challenge**: We love climbing uphill—preferably as a team, not solo. Ever tried bouldering on fossil fuels? Not recommended.  
- **We Are Stronger Together**: Like a good smoothie, we mix diverse talents to make something delicious!   
- **We Unlock Brilliance**: Not just lights in the ceiling, but also the glow of innovative ideas! 💡

Emily from HR says our secret sauce is "carrot sticks and collective brilliance!"

---

### Join Us: The Careers Section with a Twist!

Become part of a rockstar cast with *Worley*! Here’s the lineup:

- **Graduates & Early Careers**: Put your brain to work! Find yourself lost between pipelines and pizza parties while seeking sustainable transformations.  
- **Trade & Craft**: Got skills? We have jobs that require both survival skills and being able to read blueprints! It’s a thrill-a-minute!

Here’s what you get: the chance to save the planet, one construction plan at a time—and guess what? It comes with benefits too! (Including a discount on sunscreen—because we care!)

---

### Customer Testimonials: Just Kidding, Our Customers Don’t Speak—We Deliver! 🤐 

Let's be honest—our customers are busy reducing carbon footprints and nurturing their assets. They let us do the talking while we deal with everything from carbon capture to designing energy solutions, making their lives easier than finding the last piece of the jigsaw puzzle!

---

### In Conclusion: Why Join Worley?

With a portfolio as diverse as our jokes, we’re not just *doing business*, we’re innovating for a sustainable future while having fun along the way! So, if you want to work with a company that values energy, creativity, and a pinch of sarcasm, **Worley is your next stop on this merry-go-round of sustainability!** 🎠

---

**Want to leap into this exciting world?** Check out our career page or send us a carrier pigeon (or an email, PDA-style). We can't wait to connect!

---

*Discover more at [Worley.com](http://Worley.com)*!

In [27]:
stream_brochure("Worley", "https://www.worley.com/")

Found links: {'links': [{'type': 'about page', 'url': 'https://www.worley.com/en/about-us'}, {'type': 'careers page', 'url': 'https://www.worley.com/en/careers'}, {'type': 'company profile', 'url': 'https://www.worley.com/en/about-us/our-story'}, {'type': 'contact page', 'url': 'https://www.worley.com/en/contact-us'}, {'type': 'investor relations', 'url': 'https://www.worley.com/en/investor-relations'}, {'type': 'sustainability page', 'url': 'https://www.worley.com/en/sustainability'}]}



# Welcome to Worley

## About Us
At Worley, we are at the forefront of delivering sustainable change in the energy, chemicals, and resources sectors. With nearly 50,000 experts across 45 countries, we collaborate closely with our customers to bridge the gap between traditional energy solutions and a sustainable future. Our goal is to derive 75% of our revenue from sustainability-related projects by fiscal year 2026, positioning us as a leader in the rapid transition towards a lower carbon and distributed energy system.

## Our Services
Worley offers comprehensive services across various sectors, including:

- **Chemicals and Fuels:** From refining to low carbon fuels and carbon capture initiatives.
- **Conventional Energy:** Supporting integrated gas and conventional energy infrastructure.
- **Low Carbon Energy:** Including advancements in renewable energy, hydrogen, and nuclear power.
- **Resources:** Expertise in mining essential materials such as aluminum, copper, and precious metals.

Our advanced project delivery methods ensure high-quality outcomes from engineering to installation and maintenance.

## Sustainability
Worley is dedicated to sustainability, focusing on economic, social, and environmental impacts. Our commitment is reflected in our ongoing initiatives and the Worley Foundation, which has supported over 81 projects in STEM education, skilled volunteering, and community benefits over the last decade.

### Key Sustainability Metrics:
- **52%** of our revenue is derived from sustainability-related projects.
- **48%** from traditional aggregates.

## Company Culture
At Worley, our values shape our corporate culture, emphasizing:
- **Life:** Prioritizing the safety and well-being of our people and the environment.
- **Challenge:** Embracing challenges and striving for innovative solutions.
- **Collaboration:** Building relationships and recognizing that diversity strengthens us.
- **Brilliance:** Committing to continuous learning and expert sharing.

### Join Us
There are numerous opportunities to shape a sustainable future with us. Whether you're a recent graduate, in trade and craft, or an experienced professional, you can find a fulfilling career at Worley.

## Notable Projects
We take pride in our extensive portfolio, with case studies highlighting our commitment to both traditional and sustainable energy projects. Noteworthy achievements include:
- Delivering materials handling systems for Rio Tinto at Oyu Tolgoi.
- Fast-tracking essential infrastructure to support Germany’s gas supply.

## Connect with Us
Explore our career opportunities or collaborate with us in creating innovative solutions for today's and tomorrow's energy challenges.

- **Website:** [Worley.com](https://www.worley.com)
- **Contact:** [Get in Touch](https://www.worley.com/contact)

### Together, we can bridge the gap to a sustainable future.

---

**Worley**: Delivering solutions for a better tomorrow.



## Homework
#### Multi-shot prompting 

In [28]:
link_system_prompt = """
You are provided with a list of links found on a webpage. Your task is to first categorize each link into one of the following categories:
- about page
- careers page
- terms of service
- privacy policy
- contact page
- other (please specify).

Once the links are categorized, please choose which links are most relevant to include in a brochure about the company. 
The brochure should only include links such as About pages, Careers pages, or Company Overview pages. Exclude any links related to Terms of Service, Privacy Policy, or email addresses.

Respond in the following JSON format:
{
    "categorized_links": [
        {"category": "about page", "url": "https://full.url/about"},
        {"category": "careers page", "url": "https://full.url/careers"},
        {"category": "terms of service", "url": "https://full.url/terms"},
        {"category": "privacy policy", "url": "https://full.url/privacy"},
        {"category": "other", "specify": "contact page", "url": "https://full.url/contact"}
    ],
    "brochure_links": [
        {"type": "about page", "url": "https://full.url/about"},
        {"type": "careers page", "url": "https://full.url/careers"}
    ]
}

Please find the links below and proceed with the task:

Links (some may be relative links):
[INSERT LINK LIST HERE]
"""

In [29]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please categorize these links and then decide which are relevant web links for a brochure about the company. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [30]:
def get_links(url):
    website = Website(url)
    completion = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = completion.choices[0].message.content
    return json.loads(result)

In [31]:
get_links("https://worley.com")

{'categorized_links': [{'category': 'about page', 'url': '/en/about-us'},
  {'category': 'about page', 'url': '/en/about-us/our-story'},
  {'category': 'about page', 'url': '/en/about-us/our-people/andy-loose'},
  {'category': 'about page', 'url': '/en/about-us/our-people/sevi-rich'},
  {'category': 'about page', 'url': '/en/about-us/our-people/mervyn-stevens'},
  {'category': 'about page',
   'url': '/en/about-us/our-people/carola-sepulveda'},
  {'category': 'about page', 'url': '/en/about-us/our-people/daniel-cairns'},
  {'category': 'about page',
   'url': '/en/about-us/our-people/richard-fiorante'},
  {'category': 'about page', 'url': '/en/about-us/where-we-operate'},
  {'category': 'careers page', 'url': '/en/careers'},
  {'category': 'contact page', 'url': '/en/contact-us'},
  {'category': 'terms of service', 'url': '/en/site-services/privacy'},
  {'category': 'terms of service', 'url': '/en/site-services/cookie-notice'},
  {'category': 'terms of service',
   'url': '/en/site-ser

#### Create the brochure prompt add sections to the brochure

In [32]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information.\
Structure the brochure to include specific sections as follows:\
About Us\
What we do\
How We Do It\
Where We Do It\
Our People\
Our Culture\
Connect with Us"

In [44]:
from urllib.parse import urljoin

def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()  # Get the landing page content
    
    links = get_links(url)  # Retrieve the links JSON
    
    brochure_links = links.get('brochure_links', [])  # Get the brochure links list (which is already a list)
    print("Found Brochure links:", brochure_links)  # Debug output to show the brochure links
    
    # Iterate over each brochure link
    for link in brochure_links:
        result += f"\n\n{link['type']}:\n"  # Add the type of link (about page, careers page, etc.)
        
        # Handle relative URLs by converting them to absolute URLs
        full_url = urljoin(url, link["url"])
        
        # Fetch and append the content of the brochure link URL
        result += Website(full_url).get_contents()  
    
    return result


In [45]:
print(get_all_details("https://worley.com"))

Found Brochure links: [{'type': 'about page', 'url': '/en/about-us'}, {'type': 'about page', 'url': '/en/about-us/our-story'}, {'type': 'about page', 'url': '/en/about-us/our-people/andy-loose'}, {'type': 'about page', 'url': '/en/about-us/our-people/sevi-rich'}, {'type': 'about page', 'url': '/en/about-us/our-people/mervyn-stevens'}, {'type': 'about page', 'url': '/en/about-us/our-people/carola-sepulveda'}, {'type': 'about page', 'url': '/en/about-us/our-people/daniel-cairns'}, {'type': 'about page', 'url': '/en/about-us/our-people/richard-fiorante'}, {'type': 'about page', 'url': '/en/about-us/where-we-operate'}, {'type': 'careers page', 'url': '/en/careers'}]
Landing page:
Webpage Title:
Welcome to Worley - Worley

Webpage Contents:
Solutions
Industries
Chemicals and fuels
Overview
Chemicals
Low carbon fuels
Refining
Conventional energy
Overview
Carbon capture, utilization and storage
Decommissioning
Integrated gas
Midstream energy infrastructure
Offshore oil
Onshore oil
Low carbon 

#### Stream the brochure

In [46]:
stream_brochure("Worley", "https://www.worley.com/")

Found Brochure links: [{'type': 'about page', 'url': 'https://www.worley.com/en/about-us'}, {'type': 'about page', 'url': 'https://www.worley.com/en/about-us/our-story'}, {'type': 'about page', 'url': 'https://www.worley.com/en/about-us/our-people/andy-loose'}, {'type': 'about page', 'url': 'https://www.worley.com/en/about-us/our-people/sevi-rich'}, {'type': 'about page', 'url': 'https://www.worley.com/en/about-us/our-people/mervyn-stevens'}, {'type': 'about page', 'url': 'https://www.worley.com/en/about-us/our-people/carola-sepulveda'}, {'type': 'about page', 'url': 'https://www.worley.com/en/about-us/our-people/daniel-cairns'}, {'type': 'about page', 'url': 'https://www.worley.com/en/about-us/our-people/richard-fiorante'}, {'type': 'about page', 'url': 'https://www.worley.com/en/about-us/where-we-operate'}, {'type': 'careers page', 'url': 'https://www.worley.com/en/careers'}]


# Worley Brochure

## About Us
Welcome to Worley, where we are delivering sustainable change. With nearly 50,000 energy, chemicals, and resources experts across 45 countries, we are committed to bridging two worlds: transitioning to sustainable energy while providing the energy, chemicals, and resources society needs today. As a trusted engineering partner, we are focused on delivering projects and creating value throughout the lifecycle of our customers' portfolios.

## What We Do
Worley specializes in providing innovative solutions in various sectors:
- **Energy**: Supporting conventional energy while making strides toward low carbon energy systems.
- **Chemicals & Fuels**: Refining and developing chemicals for a sustainable future.
- **Resources**: Responsible mining and sourcing of essential materials for energy transition.
- **Project Delivery Services**: Comprehensive engineering, procurement, and construction solutions that cover project delivery from initiation to remediation.

## How We Do It
We utilize a data-centric approach to project delivery, ensuring that we partner closely with our customers. Our services include:
- **Engineering and Design**
- **Procurement and Supply Chain Management**
- **Construction and Fabrication**
- **Asset Performance Solutions**
- **Sustainability Initiatives** 

By integrating advanced technologies like Worley Chemetics and Worley Comprimo into our project delivery, we enhance operational efficiency and sustainability.

## Where We Do It
Our operations extend across 45 countries, engaging in diverse industries such as:
- Chemicals and Fuels
- Low Carbon Energy
- Resource Management
- Offshore and Onshore Oil
- Renewable Energy Technologies

With a robust presence globally, Worley employs localized strategies to address the unique challenges of each market.

## Our People
At Worley, our people are our greatest asset. We have an experienced team led by visionary leaders who are passionate about driving sustainable change. Notable team members include:
- **Andy Loose** - Group Vice President, LNG
- **Carola Sepulveda** - Water for Mining Lead - Peru
- **Daniel Cairns** - Senior Technical Consultant

We provide an inclusive environment where collaboration thrives, and every voice is heard.

## Our Culture
Our corporate culture is grounded in our core values:
- **We value life**
- **We rise to the challenge**
- **We are stronger together**
- **We unlock brilliance**

These principles guide our interactions with clients, communities, and each other. We're committed to fostering a diverse and innovative workplace, emphasizing safety, empowerment, and the continuous pursuit of excellence.

## Connect with Us
We invite you to explore opportunities with us or learn more about our services and values.
- **Website**: [Worley.com](https://www.worley.com)
- **Careers**: Be part of our journey toward a sustainable future.
- **Contact**: For inquiries, please reach out via our contact page on our website.

Together, let’s shape tomorrow toward a more sustainable future.

In [47]:
stream_brochure("Mace", "https://www.macegroup.com/")

Found Brochure links: [{'type': 'about page', 'url': '/about-us/our-story'}, {'type': 'about page', 'url': '/about-us/financial-performance'}, {'type': 'about page', 'url': '/about-us/esg-performance'}, {'type': 'about page', 'url': '/about-us/our-strategy'}, {'type': 'about page', 'url': '/about-us/corporate-governance'}, {'type': 'about page', 'url': '/about-us/a-responsible-business'}, {'type': 'about page', 'url': '/about-us/innovation'}, {'type': 'about page', 'url': '/about-us/health-safety-and-wellbeing'}, {'type': 'careers page', 'url': '/careers/experienced-professionals'}, {'type': 'careers page', 'url': 'https://careers.macegroup.com/gb/en/emerging-talent'}]


# Mace Group Brochure

## About Us
Mace Group is a global leader in shaping the built environment, passionately committed to redefining the boundaries of ambition. Founded 33 years ago, we believe in the transformative power of infrastructure to create a more connected, resilient, and sustainable world. Headquartered in four global hubs across Europe, the Middle East, Africa, the Americas, and Asia Pacific, we are dedicated to delivering innovative projects that enhance communities and redefine industries.

## What We Do
Mace provides a comprehensive suite of services that cover the entire property and infrastructure lifecycle:
- **Consulting Services**: Project and Programme Management, Cost Consultancy, Architecture and Design Services, Building Information Modelling (BIM), Strategic Advisory Services, and more.
- **Construction Services**: Includes Contracting, Construction Management, Interiors, and Specialist Services.
Our expertise spans diverse sectors, from healthcare and education to transport and technology, ensuring we meet the complex needs of our clients.

## How We Do It
Our approach is built on a foundation of integrity, client focus, and a commitment to safety. We embrace challenges as opportunities to innovate and create value. By fostering collaboration between our clients, partners, and communities, we deliver exceptional results that exceed expectations—connecting capabilities to drive more resilient and sustainable solutions.

## Where We Do It
Mace operates in key strategic locations worldwide:
- **UK and Europe**: London, Midlands, and South West directly to Germany, Ireland, and Spain.
- **Americas**: Including North America and Latin America.
- **Middle East and Africa**: Presence in UAE, Oman, Qatar, KSA, East Africa, South Africa, and West Africa.
- **Asia Pacific**: Operations across Hong Kong, Singapore, Vietnam, India, Philippines, and Australia/New Zealand.

## Our People
At Mace, our people are our greatest asset. We empower our workforce through continuous development and promote an inclusive environment that celebrates diversity and individuality. Whether you are an experienced professional or emerging talent, we provide a platform for growth and support to unleash your potential within the built environment.

## Our Culture
Mace is characterized by a vibrant culture that values safety, inclusivity, and a commitment to societal impact. We encourage employees to think boldly and challenge traditional norms. Our core values—Safety First, Client Focus, Create Opportunity, and Integrity—guide our actions and shape our collaborative workplace environment, fostering a culture of respect and openness.

## Connect with Us
Explore career opportunities and learn more about Mace:
- **Website**: [Mace Group](https://www.macegroup.com)
- **Follow Us**: Connect via our social media platforms: [Twitter](#), [LinkedIn](#), [YouTube](#), [Instagram](#).

Together, let’s build a sustainable future and inspire the stories that shape our world!

#### Stream the brochure in Spanish

In [49]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information.\
Structure the brochure to include specific sections as follows:\
About Us\
What we do\
How We Do It\
Where We Do It\
Our People\
Our Culture\
Connect with Us.\
Please provide two versions of the brochure, the first in English, the second in Spanish.  The contents of the brochure are to be the same for both languages."

In [50]:
stream_brochure("Mace", "https://www.macegroup.com/")

Found Brochure links: [{'type': 'about page', 'url': '/about-us/our-story'}, {'type': 'about page', 'url': '/about-us/financial-performance'}, {'type': 'about page', 'url': '/about-us/esg-performance'}, {'type': 'about page', 'url': '/about-us/our-strategy'}, {'type': 'about page', 'url': '/about-us/corporate-governance'}, {'type': 'about page', 'url': '/about-us/edi'}, {'type': 'about page', 'url': '/about-us/a-responsible-business'}, {'type': 'about page', 'url': '/about-us/innovation'}, {'type': 'careers page', 'url': '/careers/experienced-professionals'}, {'type': 'careers page', 'url': 'https://careers.macegroup.com/gb/en/emerging-talent'}]


# Brochure for Mace Group

## About Us
Mace is a leading global expert in shaping the built environment. With over 33 years of experience, we strive to redefine the boundaries of ambition by leveraging our vast expertise to help clients, communities, and society achieve their greatest potential. Our mission is to create a more connected, resilient, and sustainable world through innovative construction and consultancy solutions.

## What We Do
We provide a comprehensive range of services across the property and infrastructure lifecycle, including:
- **Consult:** Project and Programme Management, Cost Consultancy, Architecture and Design, Building Information Modelling (BIM), Strategic Advisory Services, and more.
- **Construct:** Contracting, Construction Management, Interiors, and Specialist Services.

Our work spans several sectors, including healthcare, education, transportation, energy, defense, public sector, and more.

## How We Do It
Our approach focuses on client-centric solutions, fostering innovation, and promoting responsible business practices. We are committed to sustainability and social value, aiming to exceed expectations while ensuring safety and well-being for our employees and clients.

## Where We Do It
Mace operates globally from four strategic hubs, allowing us to serve clients in diverse regions effectively. Our locations include:
- **UK and Europe:** London, Midlands, North, Germany, Ireland, Spain
- **Americas:** North America, Latin America
- **Middle East and Africa:** UAE, Oman, Qatar, Saudi Arabia, South Africa, and more
- **Asia Pacific:** Hong Kong, Singapore, Vietnam, India, Australia, and New Zealand

## Our People
Our people are our greatest asset. We empower and support our team members to embrace challenges and drive innovation. With a focus on equality, diversity, and inclusion, we create an environment where every idea and perspective can thrive. We actively seek both experienced professionals and emerging talent to join our dynamic team.

## Our Culture
At Mace, we prioritize safety, integrity, and a client-focused mentality. Our core values guide our behavior and define our corporate culture. We believe in creating opportunities for our people to excel while maintaining a commitment to sustainable and responsible business practices.

## Connect with Us
We invite you to explore our projects and services or join our team. Follow us on our social media channels for updates and insights:
- [Twitter](https://twitter.com/MaceGroup)
- [LinkedIn](https://www.linkedin.com/company/macegroup)
- [YouTube](https://www.youtube.com/MaceGroup)
- [Instagram](https://www.instagram.com/macegroup)

---

# Folleto para Mace Group

## Acerca de Nosotros
Mace es un experto global líder en la configuración del entorno construido. Con más de 33 años de experiencia, nos esforzamos por redefinir los límites de la ambición aprovechando nuestra vasta experiencia para ayudar a clientes, comunidades y sociedades a alcanzar su máximo potencial. Nuestra misión es crear un mundo más conectado, resiliente y sostenible a través de soluciones constructivas e innovadoras.

## Lo Que Hacemos
Ofrecemos una amplia gama de servicios en todo el ciclo de vida de la propiedad y la infraestructura, incluyendo:
- **Consultoría:** Gestión de Proyectos y Programas, Consultoría de Costos, Arquitectura y Diseño, Modelado de Información de Construcción (BIM), Servicios de Asesoría Estratégica, entre otros.
- **Construcción:** Contratación, Gestión de la Construcción, Interiores y Servicios Especializados.

Nuestro trabajo abarca varios sectores, incluyendo salud, educación, transporte, energía, defensa, sector público y más.

## Cómo Lo Hacemos
Nuestro enfoque se centra en soluciones centradas en el cliente, fomentando la innovación y promoviendo prácticas empresariales responsables. Nos comprometemos con la sostenibilidad y el valor social, buscando superar expectativas mientras garantizamos la seguridad y el bienestar de nuestros empleados y clientes.

## Dónde Lo Hacemos
Mace opera a nivel global desde cuatro centros estratégicos, lo que nos permite servir efectivamente a clientes en diversas regiones. Nuestras ubicaciones incluyen:
- **Reino Unido y Europa:** Londres, Midlands, Norte, Alemania, Irlanda, España
- **Américas:** América del Norte, América Latina
- **Medio Oriente y África:** Emiratos Árabes Unidos, Omán, Qatar, Arabia Saudita, Sudáfrica y más
- **Asia-Pacífico:** Hong Kong, Singapur, Vietnam, India, Australia y Nueva Zelanda

## Nuestra Gente
Nuestra gente es nuestro mayor activo. Empoderamos y apoyamos a nuestros miembros del equipo para que enfrenten desafíos y fomenten la innovación. Con un enfoque en la igualdad, diversidad e inclusión, creamos un entorno donde cada idea y perspectiva puede prosperar. Buscamos activamente tanto profesionales experimentados como talento emergente para unirse a nuestro equipo dinámico.

## Nuestra Cultura
En Mace, priorizamos la seguridad, la integridad y una mentalidad centrada en el cliente. Nuestros valores fundamentales guían nuestro comportamiento y definen nuestra cultura corporativa. Creemos en crear oportunidades para que nuestra gente sobresalga mientras mantenemos un compromiso con prácticas empresariales sostenibles y responsables.

## Conéctate Con Nosotros
Te invitamos a explorar nuestros proyectos y servicios o unirte a nuestro equipo. Síguenos en nuestras redes sociales para actualizaciones y conocimientos:
- [Twitter](https://twitter.com/MaceGroup)
- [LinkedIn](https://www.linkedin.com/company/macegroup)
- [YouTube](https://www.youtube.com/MaceGroup)
- [Instagram](https://www.instagram.com/macegroup)